<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0fed6f61334dcb37e9192669f0dfbd6b6f78d53c88025d95bffca74205b6b986
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-06 12:58:58
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: -39.21 K (-0.28%)
Current PnL: -20.79 L (-13.8%)
CY Booked + Current PnL: -9.35 L (-6.21%)
-------------------
Total profit:  1.34 L
Total loss:  -22.13 L
-------------------
Total Booked + Current PnL: 17.63 L (14.25%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.13%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.00 L (62.5%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.86%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.39,115344.0,14840.0,14268.0,-1.60,14.77,12.37,28.96,79.0,1.04,0.82,53.75,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.17,38.0,M-SC,1.53,83211.0,-17566.0,17657.0,-1.36,-17.43,21.22,0.09,245.0,-0.99,0.59,9.32,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.17,47.0,H-MC,2.88,188500.0,23830.0,19491.0,-0.75,14.47,10.34,26.31,89.0,1.22,1.34,23.61,X40N,NTT,AC
34,ICICIGI,2260.25,-17.32,58.0,H-MC,1.77,142339.0,6206.0,24924.0,0.62,4.56,17.51,22.86,91.0,0.25,1.02,17.75,X40,ATH,INSURANCE
42,ITC,452.00,-39.61,41.0,H-LC,2.05,195600.0,-4538.0,25428.0,-1.06,-2.27,13.00,10.44,4.0,-0.18,1.39,3.17,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.10,-40.28,44.0,H-SC,16.07,90540.0,-11960.0,52558.0,5.00,-11.67,58.05,39.61,114.0,-0.23,0.65,10.58,AR,ATH,MISC
48,LTIM,7230.20,-4.62,52.0,H-LC,1.99,236588.0,-9563.0,88768.0,2.69,-3.89,37.52,32.18,16.0,-0.11,1.69,30.76,X200,ATH,IT
65,SIEMENS,4671.50,1.03,57.0,H-LC,1.93,162200.0,-23895.0,71368.0,2.53,-12.84,44.00,25.51,15.0,-0.33,1.16,19.53,AR,ATH,ELECTRICAL
74,TCS,4406.34,-29.97,43.0,H-LC,12.25,279039.0,-66917.0,135166.0,2.30,-19.34,48.44,19.73,1.0,-0.50,1.99,2.77,X40,ATH,IT
4,ANGELONE,3033.00,4.08,48.0,H-SC,11.30,184270.0,-6736.0,64439.0,2.08,-3.53,34.97,30.21,157.0,-0.10,1.31,16.62,X40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-180.68,61.0,M-SC,11.38,139725.0,-9495.0,153670.0,-5.93,-6.36,109.98,96.62,208.0,-0.06,1.00,51.21,XY25,NTT,FINANCE
66,SIS,528.00,2001.04,31.0,H-SC,3.62,84788.0,-26244.0,49855.0,-3.15,-23.64,58.80,21.26,156.0,-0.53,0.60,14.56,OX40N,NTT,MISC
60,SAIL,228.00,44.35,51.0,M-MC,12.22,227425.0,2463.0,165202.0,-2.14,1.09,72.64,74.53,192.0,0.01,1.62,33.70,XY24,BTT,STEEL
24,DEN,75.00,96.89,44.0,M-SC,2.10,55200.0,-29264.0,64799.0,-2.07,-34.65,117.39,42.07,237.0,-0.45,0.39,17.23,AR,NTT,ENTERTAINMENT
46,KPIGREEN,731.05,-6.33,33.0,H-SC,7.36,111749.0,-13548.0,72480.0,-2.02,-10.81,64.86,47.03,141.0,-0.19,0.80,39.92,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.0,-21.22,48.0,H-MC,2.13,135566.0,-222.0,42974.0,-0.21,-0.16,31.70,31.48,107.0,-0.01,0.97,11.94,X40,ATH,INSURANCE
86,WIPRO,420.0,-14.70,43.0,M-LC,5.65,150077.0,-868.0,110322.0,0.45,-0.58,73.51,72.51,53.0,-0.01,1.07,5.41,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,108.87,67.0,H-SC,12.87,131733.0,-10746.0,30272.0,0.93,-7.54,22.98,13.70,163.0,-0.35,0.94,55.48,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.24,57.0,H-SC,2.01,225432.0,-44235.0,80569.0,-0.74,-16.40,35.74,13.47,138.0,-0.55,1.61,14.63,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,93.17,38.0,M-SC,1.53,83211.0,-17566.0,17657.0,-1.36,-17.43,21.22,0.09,245.0,-0.99,0.59,9.32,OX40N,NTT,DURABLES
18,CERA,9475.0,-22.67,38.0,H-SC,1.46,141002.0,-34901.0,76931.0,-0.29,-19.84,54.56,23.89,149.0,-0.45,1.01,21.77,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-24.55,42.0,H-MC,6.24,102058.0,-29777.0,72002.0,0.30,-22.59,70.55,32.03,98.0,-0.41,0.73,14.31,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1255.66,70.0,M-SC,20.18,184373.0,3230.0,42240.0,-0.07,1.78,22.91,25.10,235.0,0.08,1.31,47.76,XY24,NTT,HEALTHCARE
60,SAIL,228.00,44.35,51.0,M-MC,12.22,227425.0,2463.0,165202.0,-2.14,1.09,72.64,74.53,192.0,0.01,1.62,33.70,XY24,BTT,STEEL
32,HINDZINC,730.22,31.07,68.0,M-LC,5.58,213737.0,8661.0,103919.0,-0.03,4.22,48.62,54.89,52.0,0.08,1.52,29.34,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.17,47.0,H-MC,2.88,188500.0,23830.0,19491.0,-0.75,14.47,10.34,26.31,89.0,1.22,1.34,23.61,X40N,NTT,AC
84,VOLTAS,1530.00,-3.32,44.0,H-MC,1.62,202860.0,11118.0,26636.0,-0.59,5.80,13.13,19.69,99.0,0.42,1.45,12.99,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.39,115344.0,14840.0,14268.0,-1.60,14.77,12.37,28.96,79.0,1.04,0.82,53.75,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,67.0,H-MC,10.16,192496.0,12688.0,99732.0,-0.29,7.06,51.81,62.52,88.0,0.13,1.37,43.55,XR,NTT,BANKS
32,HINDZINC,730.22,31.07,68.0,M-LC,5.58,213737.0,8661.0,103919.0,-0.03,4.22,48.62,54.89,52.0,0.08,1.52,29.34,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.83,35.0,H-SC,9.50,125907.0,2571.0,129055.0,0.18,2.08,102.50,106.72,119.0,0.02,0.90,25.95,AR,ATH,MISC
86,WIPRO,420.00,-14.70,43.0,M-LC,5.65,150077.0,-868.0,110322.0,0.45,-0.58,73.51,72.51,53.0,-0.01,1.07,5.41,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.00,31.0,H-LC,9.77,280848.0,-34994.0,147670.0,-0.73,-11.08,52.58,35.67,5.0,-0.24,2.00,1.24,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-18.29,32.0,H-LC,5.81,206316.0,-45452.0,98660.0,-0.56,-18.05,47.82,21.13,27.0,-0.46,1.47,10.98,X40,ATH,PAINTS
42,ITC,452.00,-39.61,41.0,H-LC,2.05,195600.0,-4538.0,25428.0,-1.06,-2.27,13.00,10.44,4.0,-0.18,1.39,3.17,X40,NTT,FMCG
57,RELIANCE,1533.00,-14.79,42.0,H-LC,3.39,213392.0,3026.0,25756.0,0.33,1.44,12.07,13.68,37.0,0.12,1.52,18.20,XY25,NTT,REFINERIES
44,JIOFIN,387.00,-3.91,43.0,H-LC,12.76,261010.0,-6355.0,71804.0,0.60,-2.38,27.51,24.48,48.0,-0.09,1.86,51.29,XY24,BTT,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,1.03,57.0,H-LC,1.93,162200.0,-23895.0,71368.0,2.53,-12.84,44.00,25.51,15.0,-0.33,1.16,19.53,AR,ATH,ELECTRICAL
48,LTIM,7230.2,-4.62,52.0,H-LC,1.99,236588.0,-9563.0,88768.0,2.69,-3.89,37.52,32.18,16.0,-0.11,1.69,30.76,X200,ATH,IT
42,ITC,452.0,-39.61,41.0,H-LC,2.05,195600.0,-4538.0,25428.0,-1.06,-2.27,13.00,10.44,4.0,-0.18,1.39,3.17,X40,NTT,FMCG
51,NESTLEIND,1377.0,-10.39,51.0,H-LC,3.22,275863.0,10437.0,46345.0,0.77,3.93,16.80,21.40,11.0,0.23,1.97,10.84,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.79,42.0,H-LC,3.39,213392.0,3026.0,25756.0,0.33,1.44,12.07,13.68,37.0,0.12,1.52,18.20,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.00,31.0,H-LC,9.77,280848.0,-34994.0,147670.0,-0.73,-11.08,52.58,35.67,5.0,-0.24,2.00,1.24,X40N,ATH,FMCG
74,TCS,4406.34,-29.97,43.0,H-LC,12.25,279039.0,-66917.0,135166.0,2.30,-19.34,48.44,19.73,1.0,-0.50,1.99,2.77,X40,ATH,IT
42,ITC,452.00,-39.61,41.0,H-LC,2.05,195600.0,-4538.0,25428.0,-1.06,-2.27,13.00,10.44,4.0,-0.18,1.39,3.17,X40,NTT,FMCG
30,HCLTECH,1937.36,-2.37,44.0,H-LC,9.99,222548.0,-19376.0,81608.0,1.72,-8.01,36.67,25.73,8.0,-0.24,1.59,5.17,X40,ATH,IT
80,UNITDSPR,1644.00,-11.80,55.0,H-LC,7.00,231340.0,-2804.0,51427.0,-1.29,-1.20,22.23,20.77,86.0,-0.05,1.65,5.64,X40N,NTT,BREWERIES


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,1.03,57.0,H-LC,1.93,162200.0,-23895.0,71368.0,2.53,-12.84,44.00,25.51,15.0,-0.33,1.16,19.53,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,41.0,H-LC,2.05,195600.0,-4538.0,25428.0,-1.06,-2.27,13.00,10.44,4.0,-0.18,1.39,3.17,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.29,32.0,H-LC,5.81,206316.0,-45452.0,98660.0,-0.56,-18.05,47.82,21.13,27.0,-0.46,1.47,10.98,X40,ATH,PAINTS
19,CIPLA,1795.00,-22.52,43.0,H-LC,4.01,207117.0,2617.0,38793.0,-0.39,1.28,18.73,20.25,10.0,0.07,1.48,10.35,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-14.79,42.0,H-LC,3.39,213392.0,3026.0,25756.0,0.33,1.44,12.07,13.68,37.0,0.12,1.52,18.20,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-29.97,43.0,H-LC,12.25,279039.0,-66917.0,135166.0,2.30,-19.34,48.44,19.73,1.0,-0.50,1.99,2.77,X40,ATH,IT
40,INFY,2275.00,-19.96,45.0,H-LC,8.15,311406.0,-1482.0,173173.0,1.06,-0.47,55.61,54.87,3.0,-0.01,2.22,6.14,X40,BTT,IT
42,ITC,452.00,-39.61,41.0,H-LC,2.05,195600.0,-4538.0,25428.0,-1.06,-2.27,13.00,10.44,4.0,-0.18,1.39,3.17,X40,NTT,FMCG
83,VBL,671.64,-22.00,31.0,H-LC,9.77,280848.0,-34994.0,147670.0,-0.73,-11.08,52.58,35.67,5.0,-0.24,2.00,1.24,X40N,ATH,FMCG
1,ABB,7934.00,-40.35,46.0,H-LC,7.58,249384.0,-12235.0,131450.0,0.23,-4.68,52.71,45.57,7.0,-0.09,1.78,6.12,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,45.0,L-SC,32.81,79680.0,-33869.0,73919.0,0.65,-29.83,92.77,35.27,268.0,-0.46,0.57,95.02,XR,NTT,HOTELS
49,MASFIN,398.61,-18.70,47.0,H-SC,11.52,92250.0,-5730.0,27334.0,1.22,-5.85,29.63,22.05,152.0,-0.21,0.66,34.09,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.39,115344.0,14840.0,14268.0,-1.60,14.77,12.37,28.96,79.0,1.04,0.82,53.75,MH,ATH,METALS
78,UJJIVANSFB,60.00,108.87,67.0,H-SC,12.87,131733.0,-10746.0,30272.0,0.93,-7.54,22.98,13.70,163.0,-0.35,0.94,55.48,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-180.68,61.0,M-SC,11.38,139725.0,-9495.0,153670.0,-5.93,-6.36,109.98,96.62,208.0,-0.06,1.00,51.21,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,66.67,69.0,H-SC,5.98,152633.0,-26438.0,92160.0,-1.55,-14.76,60.38,36.71,135.0,-0.29,1.09,51.44,XR,NTT,BANKS
64,SHALBY,327.00,1255.66,70.0,M-SC,20.18,184373.0,3230.0,42240.0,-0.07,1.78,22.91,25.10,235.0,0.08,1.31,47.76,XY24,NTT,HEALTHCARE
32,HINDZINC,730.22,31.07,68.0,M-LC,5.58,213737.0,8661.0,103919.0,-0.03,4.22,48.62,54.89,52.0,0.08,1.52,29.34,X5K,ATH,METALS
61,SAMMAANCAP,326.00,-180.68,61.0,M-SC,11.38,139725.0,-9495.0,153670.0,-5.93,-6.36,109.98,96.62,208.0,-0.06,1.00,51.21,XY25,NTT,FINANCE
11,BANKINDIA,190.00,-25.47,67.0,H-MC,10.16,192496.0,12688.0,99732.0,-0.29,7.06,51.81,62.52,88.0,0.13,1.37,43.55,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.05
1,25,43.23
2,50,74.89


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.19
LC    32.10
MC    23.71
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.90
X40      14.56
X40N     11.20
XR       11.04
XY25     10.33
AR        8.24
OX40N     7.75
X200      1.69
MH        1.62
X5K       1.52
SR        1.15
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.08
H-LC    25.58
H-MC    20.48
M-SC    13.68
M-LC     5.52
M-MC     2.93
L-SC     1.43
L-LC     1.00
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.79,-6.08,41.05
IT,12.66,-18.58,82.81
FINANCE,10.42,-15.49,67.94
MISC,6.97,-17.71,82.94
BANKS,6.37,-11.55,70.23
PAINTS,5.56,-18.47,36.13
ELECTRICAL,5.36,-9.66,48.13
INSURANCE,3.77,-4.85,44.05
AUTO,3.53,-16.15,71.68


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3238135.0,22
XR,1361649.0,14
AR,1176696.0,9
X40,798895.0,10
X40N,634541.0,9
OX40N,559957.0,10
XY25,489558.0,7
SR,260928.0,2
X5K,103919.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3716212.0,29
M-SC,1512218.0,17
H-MC,1234304.0,15
H-LC,1226102.0,15
M-LC,374174.0,4
M-MC,371660.0,2
L-SC,261160.0,3
L-MC,60450.0,1
L-LC,43514.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1219449.0      6
M-SC       XY24       846214.0      7
H-SC       AR         805809.0      5
           XR         788637.0      7
H-MC       XY24       569075.0      4
H-LC       X40        514035.0      5
M-MC       XY24       371660.0      2
H-SC       X40N       310893.0      4
           SR         260928.0      2
           OX40N      258016.0      4
H-LC       X40N       237890.0      3
H-MC       X40        211882.0      4
H-LC       AR         202818.0      2
H-MC       XY25       181587.0      2
L-SC       XR         172371.0      2
M-SC       AR         168069.0      2
M-LC       XY24       159933.0      2
M-SC       XY25       153670.0      1
           OX40N      141150.0      4
           XR         130137.0      2
H-LC       XY25       110787.0      3
M-LC       XR         110322.0      1
           X5K        103919.0      1
H-MC       XR          99732.0      1
L-SC       OX40N       88789.0      1
H-LC       X200        88768.0      1
H-MC       X40N        85758.0      2
M-SC       X40         72978.0      1
H-SC       MH          72480.0      1
H-MC       OX40N       72002.0      1
H-LC       XY24        71804.0      1
L-MC       XR          60450.0      1
L-LC       XY25        43514.0      1
H-MC       MH          14268.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
